In [1]:
import numpy as np
import pandas as pd
import random
from collections import deque, defaultdict

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

In [2]:
import sys
sys.path.append('..')
import datetime
import time
import random
from collections import Counter, defaultdict, deque
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
from kneed import KneeLocator
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score, log_loss, roc_auc_score
import json
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from IPython.display import display, HTML

# display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

In [3]:
def cross_entropy_loss(model, x_test, y_test):
    """
    For each sample i:
      loss_i = -∑_c [1{c = y_true_i} · log P_model(c | x_i)]
    If the true label isn’t in model.classes_, returns a default high loss.
    Works for any len(x_test) >= 1, including the single-class case.
    """
    probs = model.predict_proba(x_test)
    default = log_loss([[1, 0]], [[0, 1]]) + 1  # fallback loss

    losses = []
    for i, true_label in enumerate(y_test):
        sample_probs = probs[i]
        classes = model.classes_

        # if only one class in the model
        if sample_probs.size == 1:
            if classes[0] == true_label:
                losses.append(0.0)  # perfect prediction
            else:
                losses.append(default)
            continue

        # find index of the true label
        idx_arr = np.where(classes == true_label)[0]
        if idx_arr.size == 0:
            losses.append(default)
        else:
            y_true_onehot = np.zeros_like(sample_probs)
            y_true_onehot[idx_arr[0]] = 1

            # normalize just in case
            sample_probs = sample_probs / sample_probs.sum()
            y_true_onehot = y_true_onehot / y_true_onehot.sum()

            loss_i = log_loss([y_true_onehot], [sample_probs])
            losses.append(loss_i)

    return np.array(losses)


In [4]:
def normal_loss(model, x_test, y_test):
    """
    For each sample i:
      loss_i = 1 - P_model(y_true_i | x_i)
    If the true label isn’t in model.classes_, we return 1.1 as before.
    Works for any len(x_test) >= 1.
    """
    # predict_proba returns shape (n_samples, n_classes)
    probs = model.predict_proba(x_test)
    
    losses = []
    for i, true_label in enumerate(y_test):
        sample_probs = probs[i]
        # find index of the true label in model.classes_
        idx_arr = np.where(model.classes_ == true_label)[0]
        if idx_arr.size == 0:
            losses.append(1.1)
        else:
            col_index = idx_arr[0]
            losses.append(1 - sample_probs[col_index])
    
    return np.array(losses)

In [5]:
def safe_transform_target(encoder, targets, unknown_value=-1):
    classes = set(encoder.classes_)
    transformed = []
    for t in targets:
        if t in classes:
            transformed.append(encoder.transform([t])[0])
        else:
            transformed.append(unknown_value)
    return np.array(transformed)

In [6]:
def get_clean_loss(normal_loss_value, cross_entropy_loss_value):
    normal_loss_dist = []
    cross_loss_dist = []
    for pos, prediction in  enumerate(normal_loss_value):
        if prediction != 1:
            cross_loss_dist.append(cross_entropy_loss_value[pos])
            normal_loss_dist.append(prediction)

    return np.array(normal_loss_dist), np.array(cross_loss_dist)

In [7]:
def cleaning_cls_result(classification_result):
    
    for i in classification_result.keys():
        print(i, classification_result[i].keys())

        if '1' not in classification_result[i].keys():
            classification_result[i]['1'] = {'precision': 0, 'recall': 0, 'f1-score': 0, 'support': 0.0}
    return classification_result

In [8]:
def find_largest_gap(losses):
    y = sorted(losses, reverse=True)
    diffs = abs(np.diff(y))
    idx = np.argmax(diffs) + 1   # +1 because diffs[i] = y[i+1]-y[i]
    return idx, y[idx]

In [9]:
# ----------------------------
# 1) Read and Process the Data
# ----------------------------
dataset = '0.099_noise.csv'
df = pd.read_csv(f"../data/{dataset}")
df = df.sort_values(by='Timestamp')
# Process 'noise' column: NaN→0, True/1/'True'/'true'→1, else 0
df['noise'] = df['noise'].fillna(0).apply(lambda x: 1 if (x is True or x == 1 or x == 'True' or x == 'true') else 0)
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.rename(columns={'Case ID': 'ID'})
print(f"Loaded: {dataset} (n_rows={len(df)})")

# ----------------------------
# 2) Prefixes & Global Window Settings
# ----------------------------
prefix_range = range(2, 38)   # 2..15
WINDOW_EVENTS = 2500          # keep last 2500 prefix‐events


Loaded: 0.099_noise.csv (n_rows=79441)


In [10]:

# ----------------------------
# 3) Pre‐fit Encoders for NAP
# ----------------------------
all_acts = df["Activity"].unique()
ohe_nap = {
    p: OneHotEncoder(sparse_output=False, handle_unknown="ignore")
         .fit(np.array([[a] * (p - 1) for a in all_acts]))
    for p in prefix_range
}
le_nap = {p: LabelEncoder().fit(all_acts) for p in prefix_range}


In [11]:

# ----------------------------
# 4) Buffers & State Initialization
# ----------------------------
# Global sliding window of the last WINDOW_EVENTS prefix‐events
global_events = deque(maxlen=WINDOW_EVENTS)

# Per‐prefix buffers (unbounded; we’ll evict manually)
buffers = {}
for p in prefix_range:
    buffers[p] = {
        "raw_feats":      deque(),
        "raw_tgts":       deque(),
        "X":               deque(),
        "y":               deque(),
        "noise":           deque(),
        "model":          None,
        "filled":         False,
        "cutoff":         None
    }

case_events      = defaultdict(list)
detect_pool      = []   # list of dicts for AD training
anom_clf         = None
enc_ad           = None
max_prob_ad      = 0
max_pfx          = max(prefix_range) - 1  # for padding raw_feats

online_nap_reports = []
online_ad_reports  = []

# Build a vocabulary (activity→index) for LSTM sequencing
act2idx = {act: i + 1 for i, act in enumerate(all_acts)}
vocab_size = len(act2idx) + 1  # +1 for padding index=0

# ----------------------------
# 4.5) Helper: compute gap‐based CE cutoff
# ----------------------------
def compute_gap_cutoff(rf, Xw, yw):
    ce_losses = cross_entropy_loss(rf, Xw, yw)
    n = ce_losses.size
    if n == 0:
        return 0.0
    if n == 1:
        return float(ce_losses[0])
    # find_largest_gap returns (idx, cutoff)
    _, cutoff_gap = find_largest_gap(ce_losses)
    return cutoff_gap

# ----------------------------
# 4.6) Define LSTM‐based AD Classifier
# ----------------------------
class AD_LSTM(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_num_feats):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            num_layers=1,
            batch_first=True
        )
        self.dropout_seq = nn.Dropout(0.2)
        self.fc_num = nn.Sequential(
            nn.Linear(num_num_feats, 64),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        self.fc_combined = nn.Sequential(
            nn.Linear(hidden_dim + 64, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x_seq, x_num):
        # x_seq: (batch, seq_len), x_num: (batch, num_feats)
        emb = self.embedding(x_seq)              # (batch, seq_len, embed_dim)
        _, (h_n, _) = self.lstm(emb)             # h_n: (1, batch, hidden_dim)
        h_last = h_n[0]                          # (batch, hidden_dim)
        h_last = self.dropout_seq(h_last)

        n_out = self.fc_num(x_num)               # (batch, 64)
        combined = torch.cat([h_last, n_out], dim=1)  # (batch, hidden_dim+64)
        return self.fc_combined(combined)        # (batch,1) sigmoid


In [12]:

# ----------------------------
# 5) Streaming Loop with NAP + LSTM‐AD
# ----------------------------
total = len(df)
global_update_counter = 0
global_retrain_batch = WINDOW_EVENTS // 2  # 1250

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for i, (_, row) in enumerate(df.iterrows(), start=1):
    # Progress logging
    if i % 1000 == 0 or i == total:
        pct = i / total * 100
        print(f"Processed {i}/{total} rows ({pct:.1f}%)")

    cid = row["ID"]
    case_events[cid].append(row)
    cur_len = len(case_events[cid])

    # Only process when a case first reaches prefix length p
    for p in prefix_range:
        if cur_len != p:
            continue

        # Build current sample
        group      = case_events[cid]
        feats      = [e.Activity for e in group[: p - 1]]
        target_act = group[p - 1].Activity
        noise_flag = group[p - 1].noise

        buf = buffers[p]

        # --- 5.1) Slide the global window (peek dropped if full) ---
        dropped = None
        if len(global_events) == WINDOW_EVENTS:
            dropped = global_events[0]  # will be popped on append()

        # Transform for NAP
        Xp_vec = ohe_nap[p].transform([feats]).ravel()
        yp     = le_nap[p].transform([target_act])[0]

        # Append to global_events: (prefix, X_vec, y_label, noise, raw_feats, raw_target)
        global_events.append((p, Xp_vec, yp, noise_flag, feats, target_act))

        # Append to this prefix’s buffers
        buf["raw_feats"].append(feats)
        buf["raw_tgts"].append(target_act)
        buf["X"].append(Xp_vec)
        buf["y"].append(yp)
        buf["noise"].append(noise_flag)

        # Evict from old prefix buffer if needed
        if dropped is not None:
            old_p, old_Xp, old_yp, old_noise, old_feats, old_tgt = dropped
            old_buf = buffers[old_p]
            if old_buf["X"]:
                old_buf["raw_feats"].popleft()
                old_buf["raw_tgts"].popleft()
                old_buf["X"].popleft()
                old_buf["y"].popleft()
                old_buf["noise"].popleft()

        # --- 5.2) Initial NAP training for prefix p ---
        if buf["model"] is None:
            Xw = np.vstack(buf["X"])
            yw = np.array(buf["y"])

            rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
            rf.fit(Xw, yw)

            cutoff = compute_gap_cutoff(rf, Xw, yw)
            buf["model"]  = rf
            buf["filled"] = True
            buf["cutoff"] = cutoff

            # Bootstrap AD pool with up to 20 samples (≤10 anomalies)
            anom_idxs = [idx for idx, flag in enumerate(buf["noise"]) if flag == 1]
            norm_idxs = [idx for idx, flag in enumerate(buf["noise"]) if flag == 0]

            n_anom = min(10, len(anom_idxs))
            sel_anom = random.sample(anom_idxs, n_anom) if n_anom > 0 else []
            sel_norm = random.sample(norm_idxs, 20 - n_anom) if len(norm_idxs) >= (20 - n_anom) else norm_idxs

            sel_idxs = sel_anom + sel_norm
            random.shuffle(sel_idxs)

            for idx in sel_idxs:
                prob_vec = rf.predict_proba(buf["X"][idx].reshape(1, -1))[0].tolist()
                ce0      = cross_entropy_loss(rf, buf["X"][idx].reshape(1, -1), [buf["y"][idx]])[0]
                detect_pool.append({
                    "raw_feats": buf["raw_feats"][idx],
                    "prefix":    p,
                    "target":    buf["raw_tgts"][idx],
                    "prob":      prob_vec,
                    "ce_loss":   ce0,
                    "anomaly":   buf["noise"][idx]
                })

            # Train LSTM‐AD if we have ≥20 samples
            if len(detect_pool) >= 20:
                # 1) Build categorical rows for OneHotEncoder (if needed)
                cat_rows = []
                for d in detect_pool:
                    row_cat = d["raw_feats"] + [None] * (max_pfx - len(d["raw_feats"]))
                    row_cat += [d["prefix"], d["target"]]
                    cat_rows.append(row_cat)
                enc_ad = OneHotEncoder(sparse_output=False, handle_unknown="ignore").fit(cat_rows)
                X_cat_np = enc_ad.transform(cat_rows)  # shape (N, D_cat)

                # 2) Build numeric features: (prob_vector + [ce_loss])
                max_prob_ad = max(len(d["prob"]) for d in detect_pool)
                prob_mat = [
                    d["prob"] + [0.0] * (max_prob_ad - len(d["prob"]))
                    for d in detect_pool
                ]
                ce_vec = [[d["ce_loss"]] for d in detect_pool]
                X_num_np = np.hstack([prob_mat, ce_vec])  # (N, max_prob_ad+1)

                # 3) Build LSTM sequences
                seqs = []
                for d in detect_pool:
                    seq = [act2idx[a] for a in d["raw_feats"]]
                    seqs.append(torch.tensor(seq, dtype=torch.long))
                X_seq = pad_sequence(seqs, batch_first=True, padding_value=0)  # (N, max_seq_len)

                # 4) Labels
                y_ad_np = np.array([d["anomaly"] for d in detect_pool])

                # 5) Create PyTorch Dataset & DataLoader
                class AD_LSTMDataset(Dataset):
                    def __init__(self, X_seq, X_num, y):
                        self.X_seq = X_seq       # LongTensor
                        self.X_num = torch.from_numpy(X_num).float()
                        self.y     = torch.from_numpy(y).float().unsqueeze(1)

                    def __len__(self):
                        return self.X_seq.shape[0]

                    def __getitem__(self, idx):
                        return self.X_seq[idx], self.X_num[idx], self.y[idx]

                dataset_lstm = AD_LSTMDataset(X_seq, X_num_np, y_ad_np)
                loader_lstm  = DataLoader(dataset_lstm, batch_size=64, shuffle=True)

                # 6) Instantiate and train the LSTM model
                embed_dim   = 32
                hidden_dim  = 64
                num_num_feats = X_num_np.shape[1]

                model_lstm = AD_LSTM(vocab_size, embed_dim, hidden_dim, num_num_feats).to(device)
                optimizer = optim.Adam(model_lstm.parameters(), lr=1e-3)
                criterion = nn.BCELoss()

                model_lstm.train()
                for epoch in range(5):  # 5 epochs for simplicity
                    total_loss = 0.0
                    for Xseq_batch, Xnum_batch, y_batch in loader_lstm:
                        Xseq_batch = Xseq_batch.to(device)
                        Xnum_batch = Xnum_batch.to(device)
                        y_batch    = y_batch.to(device)

                        optimizer.zero_grad()
                        outputs = model_lstm(Xseq_batch, Xnum_batch)
                        loss = criterion(outputs, y_batch)
                        loss.backward()
                        optimizer.step()
                        total_loss += loss.item() * Xseq_batch.size(0)

                    total_loss /= len(dataset_lstm)
                    # print(f"Epoch {epoch+1}, Loss={total_loss:.4f}")

                anom_clf = model_lstm.eval()  # set to eval mode

            continue  # skip further processing of this event

        # --- 5.3) Prequential NAP Prediction & Store ---
        rf   = buf["model"]
        Xp   = Xp_vec.reshape(1, -1)
        y_sp = yp
        cutoff_nap = buf["cutoff"]

        ce_cur = cross_entropy_loss(rf, Xp, [y_sp])[0]
        pred_nap_anom = int(ce_cur > cutoff_nap)

        online_nap_reports.append({
            "i":             i,
            "prefix":        p,
            "case_id":       cid,
            "true_noise":    noise_flag,
            "pred_nap_anom": pred_nap_anom,
            "cutoff":        cutoff_nap
        })

        # --- 5.4) Global Retrain Trigger (increment once per prefix-event) ---
        global_update_counter += 1
        if global_update_counter >= global_retrain_batch:
            for q in prefix_range:
                buf_q = buffers[q]
                if len(buf_q["X"]) == 0:
                    continue

                Xw = np.vstack(buf_q["X"])
                yw = np.array(buf_q["y"])
                rf_q = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
                rf_q.fit(Xw, yw)
                buf_q["model"] = rf_q

                cutoff_q = compute_gap_cutoff(rf_q, Xw, yw)
                buf_q["cutoff"] = cutoff_q

                # Sample up to 20 events from this prefix buffer for AD pool
                window_size = len(buf_q["y"])
                if window_size < 20:
                    sample_idxs = list(range(window_size))
                else:
                    sample_idxs = random.sample(range(window_size), k=20)

                for idx in sample_idxs:
                    prob_vec = rf_q.predict_proba(buf_q["X"][idx].reshape(1, -1))[0].tolist()
                    ce0      = cross_entropy_loss(rf_q, buf_q["X"][idx].reshape(1, -1), [buf_q["y"][idx]])[0]
                    detect_pool.append({
                        "raw_feats": buf_q["raw_feats"][idx],
                        "prefix":    q,
                        "target":    buf_q["raw_tgts"][idx],
                        "prob":      prob_vec,
                        "ce_loss":   ce0,
                        "anomaly":   buf_q["noise"][idx]
                    })

            # Retrain LSTM‐AD if detect_pool has ≥20
            if len(detect_pool) >= 20:
                # Build categorical rows & OneHotEncoder
                cat_rows = []
                for d in detect_pool:
                    row_cat = d["raw_feats"] + [None] * (max_pfx - len(d["raw_feats"]))
                    row_cat += [d["prefix"], d["target"]]
                    cat_rows.append(row_cat)
                enc_ad = OneHotEncoder(sparse_output=False, handle_unknown="ignore").fit(cat_rows)
                X_cat_np = enc_ad.transform(cat_rows)

                # Numeric features
                max_prob_ad = max(len(d["prob"]) for d in detect_pool)
                prob_mat = [
                    d["prob"] + [0.0] * (max_prob_ad - len(d["prob"]))
                    for d in detect_pool
                ]
                ce_vec = [[d["ce_loss"]] for d in detect_pool]
                X_num_np = np.hstack([prob_mat, ce_vec])

                # Build LSTM sequences
                seqs = []
                for d in detect_pool:
                    seq = [act2idx[a] for a in d["raw_feats"]]
                    seqs.append(torch.tensor(seq, dtype=torch.long))
                X_seq = pad_sequence(seqs, batch_first=True, padding_value=0)

                # Labels
                y_ad_np = np.array([d["anomaly"] for d in detect_pool])

                # Dataset & DataLoader
                dataset_lstm = AD_LSTMDataset(X_seq, X_num_np, y_ad_np)
                loader_lstm  = DataLoader(dataset_lstm, batch_size=64, shuffle=True)

                # Retrain LSTM model
                embed_dim     = 32
                hidden_dim    = 64
                num_num_feats = X_num_np.shape[1]

                model_lstm = AD_LSTM(vocab_size, embed_dim, hidden_dim, num_num_feats).to(device)
                optimizer = optim.Adam(model_lstm.parameters(), lr=1e-3)
                criterion = nn.BCELoss()

                model_lstm.train()
                for epoch in range(5):
                    total_loss = 0.0
                    for Xseq_batch, Xnum_batch, y_batch in loader_lstm:
                        Xseq_batch = Xseq_batch.to(device)
                        Xnum_batch = Xnum_batch.to(device)
                        y_batch    = y_batch.to(device)

                        optimizer.zero_grad()
                        outputs = model_lstm(Xseq_batch, Xnum_batch)
                        loss = criterion(outputs, y_batch)
                        loss.backward()
                        optimizer.step()
                        total_loss += loss.item() * Xseq_batch.size(0)

                    total_loss /= len(dataset_lstm)
                    # print(f"(Retrain) Epoch {epoch+1}, Loss={total_loss:.4f}")

                anom_clf = model_lstm.eval()  # switch to eval mode

            global_update_counter = 0

        # --- 5.5) Prequential LSTM‐AD Classification for Current Event ---
        if anom_clf is not None:
            # Build sequence for current event
            seq_new = [act2idx[a] for a in feats]
            X_seq_new = pad_sequence([torch.tensor(seq_new, dtype=torch.long)],
                                     batch_first=True, padding_value=0).to(device)
            # Build numeric for current event
            pvec_new = rf.predict_proba(Xp)[0].tolist()
            pvec_new += [0.0] * (max_prob_ad - len(pvec_new))
            ce0_new = ce_cur
            X_num_new = torch.tensor([pvec_new + [ce0_new]], dtype=torch.float).to(device)

            with torch.no_grad():
                prob_ad = float(model_lstm(X_seq_new, X_num_new).item())
            pred_ad = int(prob_ad > 0.5)

            online_ad_reports.append({
                "i":            i,
                "prefix":       p,
                "case_id":      cid,
                "true_noise":   noise_flag,
                "pred_ad_anom": pred_ad,
                "score":        prob_ad
            })




Processed 1000/79441 rows (1.3%)
Processed 2000/79441 rows (2.5%)
Processed 3000/79441 rows (3.8%)
Processed 4000/79441 rows (5.0%)
Processed 5000/79441 rows (6.3%)
Processed 6000/79441 rows (7.6%)
Processed 7000/79441 rows (8.8%)
Processed 8000/79441 rows (10.1%)
Processed 9000/79441 rows (11.3%)
Processed 10000/79441 rows (12.6%)
Processed 11000/79441 rows (13.8%)
Processed 12000/79441 rows (15.1%)
Processed 13000/79441 rows (16.4%)
Processed 14000/79441 rows (17.6%)
Processed 15000/79441 rows (18.9%)
Processed 16000/79441 rows (20.1%)
Processed 17000/79441 rows (21.4%)
Processed 18000/79441 rows (22.7%)
Processed 19000/79441 rows (23.9%)
Processed 20000/79441 rows (25.2%)
Processed 21000/79441 rows (26.4%)
Processed 22000/79441 rows (27.7%)
Processed 23000/79441 rows (29.0%)
Processed 24000/79441 rows (30.2%)
Processed 25000/79441 rows (31.5%)
Processed 26000/79441 rows (32.7%)
Processed 27000/79441 rows (34.0%)
Processed 28000/79441 rows (35.2%)
Processed 29000/79441 rows (36.5%)
P

In [14]:
# 6) Summarize
reports_df = pd.DataFrame(online_ad_reports)
for p in prefix_range:
    sub = reports_df[reports_df["prefix"] == p]
    if not sub.empty:
        print(f"\n--- Prefix {p} ---")
        print(classification_report(
            sub["true_noise"], sub["pred_ad_anom"], zero_division=0
        ))
reports_df 
reports_df.to_csv('../result/%s_classifier_lstm.csv'%(dataset), index=False)


--- Prefix 2 ---
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      4460
           1       0.87      0.95      0.91       537

    accuracy                           0.98      4997
   macro avg       0.93      0.97      0.95      4997
weighted avg       0.98      0.98      0.98      4997


--- Prefix 3 ---
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      4547
           1       0.84      0.91      0.87       450

    accuracy                           0.98      4997
   macro avg       0.91      0.95      0.93      4997
weighted avg       0.98      0.98      0.98      4997


--- Prefix 4 ---
              precision    recall  f1-score   support

           0       0.99      0.92      0.95      4526
           1       0.54      0.90      0.68       471

    accuracy                           0.92      4997
   macro avg       0.77      0.91      0.82      4997
weighted avg       0